* 在[attention_model](../apss/nets/attention_model.py)#Line226 和[attention_model_v2](../apss/nets/attention_model_v2.py)#Line227 中有一个Mindspore动态图下随机算子`ops.multinomial`内存泄漏的BUG,以下的Code是一个可复现的例子：
* 同时，如果我们想在初始化类的时候定义`self._multinomial = ops.Multinomial()`，来缓解缓存未命中的问题，但此算子没有反向。


In [ ]:
import os
import psutil
import numpy as np
from typing import Tuple
import mindspore as ms
from mindspore import nn, Tensor, ops

# 查询当前进程的内存，单位为MB
def get_cpu_memory(pid=os.getpid()):
    p = psutil.Process(pid)
    return p.memory_info().rss / (1024 * 1024)

class TestNet(nn.Cell):
    def __init__(self):
        super(TestNet, self).__init__()
        self._multinomial = ops.operations.Multinomial(123, 321)

    # sample from x with shape b*c and generate result with shape b*1
    def construct(self, x: Tensor) -> Tuple[Tensor]:
        # 使用ops.multinomial函数式调用，存在泄露
        action = ops.multinomial(x, num_sample=1)

        # 提前在构造函数创建算子实例，存在泄露
        # action = self._multinomial(x, 1)

        # 使用ops.zeros创建相同大小的输出结果，不会泄露
        # action = ops.zeros((x.shape[0], 1), ms.int32)

        return action

def test_mem_leak():
    bs = 128
    c = 1024
    net = TestNet()
    x = Tensor(np.random.rand(bs, c), ms.float32)

    # warmup
    for i in range(50):
        y = net(x)
        # 通过转换成numpy矩阵确认计算已经完成
        y_arr = y.asnumpy()

    mem0 = get_cpu_memory()
    mem1 = 0
    for i in range(2000):
        y = net(x)
        # 通过转换成numpy矩阵确认计算已经完成
        y_arr = y.asnumpy()

        del y
        del y_arr

        mem1 = get_cpu_memory()
        
        if (i+1) % 100 == 0:
            print('Step {}, cpu_mem {}MB -> {}MB'.format(i, mem0, mem1))

    print('All done: cpu_mem {}MB -> {}MB'.format(mem0, mem1))

if __name__ == '__main__':
    ms.context.set_context(device_target='GPU', mode=ms.context.PYNATIVE_MODE)
    # ms.context.set_context(device_target='GPU', mode=ms.context.GRAPH_MODE)

    test_mem_leak()